In [1]:
import csv
from pathlib import Path
import datetime

In [2]:
import time

### Configクラス 

In [9]:
class Config():
    """
    各クラス・関数が参照する変数をまとめたクラス
    """
    def __init__(self):
        self.parent = None

In [10]:
config = Config()

### 各種クロージャ 

#### enableクロージャ 

In [77]:
class MultiClosier():
    """
    複数のカウンタを利用する時に，このクロージャの終了時にまとえてtempfileを削除するためのクロージャ
    """
    def __init__(self, parent_restart):
        self.parent_restart = parent_restart
        
    def __enter__(self):
        if config.parent is not None:
            raise Exception("MultiCount has already opend. cannot open another MultiCount")
        
        config.parent = self.parent_restart
        return self
        
    def __exit__(self, ex_type, ex_value, trace):
        config.parent = None  # 共通して行う
        if ex_type is None:# 正常終了した場合
            self.parent_restart.all_close()
        return False

#### counterクロージャ 

In [65]:
class CounterClosier():
    """
    イテレーションの進捗をtempfileに保存するクロージャ，自身によってイテレータをラップする．
    途中で例外によって終了した場合と親が存在する場合にファイルを残す．
    """
    def __init__(self, child_restart, each_save=False):
        self.child_restart = child_restart
        self.each_save = each_save
        
        # 一時ファイルの読み込み
        self.start_counter = self._read()
        self.counter = 0  # 一応こちらでも0に初期化
        
    def _read(self):
        if self.child_restart.file_path.exists():
            with open(self.child_restart.file_path, "r") as f:
                reader = csv.reader(f)
                #dateについて取得, 現在時間との差が一日以内かどうか判定
                datetime_list = next(reader)  # [datetime,実際の日時の文字列]
                tempfile_datetime = datetime.datetime.strptime(datetime_list[1], "%Y-%m-%d %H:%M:%S")
                if datetime.datetime.now() - tempfile_datetime >= datetime.timedelta(days=1):
                    print("tempfile is not recent date, please check tempfile")

                # スタートカウンターの読み込み
                start_counter_list = next(reader)
                start_counter = int(start_counter_list[1])
        else:
            start_counter = 0
        
        return start_counter
    
    def _iter_finish(self):
        """
        イテレーションが終了したときの処理
        """
        if self.child_restart.parent is None:  # ペアレントが無い場合
            if self.child_restart.file_path.exists():
                self.child_restart.file_path.unlink()
        else:  # ペアレントが存在する場合
            self._write()
    
    def __call__(self, iterable):  # ジェネレーターを返す
        iterable = iter(iterable)
        self.counter = 0  # カウンタの初期化
        while True:
            if self.counter < self.start_counter:
                self.counter += 1
                try:
                    next(iterable)  # 利用しない．進めるだけ
                except StopIteration:
                    self._iter_finish()
                    return None  # StopIterationで終了
                continue
                
            
            
            try:
                yield_item = next(iterable)  # iterableから一つ取得
                yield yield_item
            except StopIteration:
                self._iter_finish()
                return None  # StopIterationで終了
            
            self.counter += 1  # すべてが終了したら+1
            if self.each_save:  # 毎回一時ファイルを保存
                self._write()
            
    
    def __enter__(self):
        return self
    
    def _write(self):
        with open(self.child_restart.file_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["datetime", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
            writer.writerow(["start_count",self.counter])
                    
    def __exit__(self, ex_type, ex_value, trace):
        if ex_type is not None:  # エラーで終了した場合 
            self._write()
        return False

In [66]:
class CounterClosierThrough():
    """
    CounterClosierを模した何もしないクロージャ．実装を変えずにtempfileの使用・不使用を切り替えるために利用する
    """
    def __init__(self, child_restart, each_save=False):
        self.child_restart = child_restart
        self.each_save = each_save
        
        #ファイルが存在したら削除
        if self.child_restart.file_path.exists():
            self.child_restart.file_path.unlink()
    
    def __call__(self, iterable):
        return iterable
    
    def __enter__(self):
        return self

    def __exit__(self, ex_type, ex_value, trace):
        return False

### RestartChild 

In [67]:
class RestartChild():
    """
    子供としてCounterClosierを渡すためのクラス
    """
    def __init__(self, file_path, parent, use_tempfile=True):
        self.file_path = Path(file_path)
        self.use_tempfile = use_tempfile
        self.parent = parent
        
    def enable(self):
        if self.use_tempfile:
            return CounterClosier(self, each_save=False)  # 終了時に保存
        else:
            return CounterClosierThrough(self)
        
    def enable_without_with(self):
        if self.use_tempfile:
            return CounterClosier(self, each_save=True)
        else:
            return CounterClosierThrough(self)

### RestartParent 

In [68]:
class RestartParent():
    """
    config.parentを変更し，親から子供を作るためにクラス
    """
    def __init__(self):
        self.child_list = []
        
    def create_child(self, file_path, use_tempfile=True):
        """
        親から子供を作成する
        """
        restart_child = RestartChild(file_path, self, use_tempfile)
        self.child_list.append(restart_child)
        return restart_child
        
    def multi_child(self):
        return MultiClosier(self)
    
    def all_close(self):
        """
        子のtempfileを削除する．
        """
        [child.file_path.unlink() for child in self.child_list if child.file_path.exists()]  # フォイルが存在している場合は，削除

### インターフェースとなる関数 

In [69]:
def multi_count():
    """
    複数カウンタを作成するときに展開することで，tempfileの削除をすべての終了タイミングに変更できる．
    """
    parent = RestartParent()
    return parent.multi_child()

def enable_counter(file_path, use_tempfile=True):
    if config.parent is None:  #ペアレントが存在しない場合
        child = RestartChild(file_path, None, use_tempfile=use_tempfile)
        return child.enable()
    
    else:
        child = config.parent.create_child(file_path)
        return child.enable()
    
def simple_counter(file_path, iterable, use_tempfile=True):
    #ジェネレータを返す
    if config.parent is None:  #ペアレントが存在しない場合
        child = RestartChild(file_path, None, use_tempfile=use_tempfile)
        return child.enable_without_with()(iterable)
    
    else:
        child = config.parent.create_child(file_path)
        return child.enable_without_with()(iterable)
    

### テストコード 

#### 一つの場合 

以下のように，`enable_counter`をwith文に添えた返り値でイテレーターをラップする．イテレーション内でエラーが生じた場合に，一時ファイルを保存し，次回はエラーが起きたイテレーションから再開できる．
この例では，iが4のときにKeybordInterruptを行った後，もう一度実行した結果である．

In [72]:
tempfile_path = Path("temp1.tmp")

with enable_counter(tempfile_path) as counter:
    for i in counter(range(10)):
        print(i)
        time.sleep(3)

4
5
6
7
8
9


#### 一つの場合(毎回保存する場合) 

with文を利用したくない場合，`simple_couonter`が利用できる．`simple_counter`は直接ジェネレータを出力するが，毎回一時ファイルを更新し，保存する．

In [75]:
tempfile_path = Path("temp2.tmp")

for i in simple_counter(tempfile_path, range(10)):
    print(i)
    time.sleep(3)

4
5
6
7
8
9


#### 二つ以上の場合 

`enable_counter`あるいは`simple_counter`のみでは，一つのfor文が終了したときに一時ファイルが削除されてしまうため二つ以上for文が連続する場合に進捗を保存できない．`multi_count{を利用すればそのインデントブロック内のすべてのでラップしたイテレーションが終了するまで一時ファイルを残すことができる．以下の例では，一つ目のfor文が終了したのちにiが2の時点でKeybordInterruptを行い，再度実行した結果である

In [81]:
tempfile_path1 = Path("temp3.tmp")
tempfile_path2 = Path("temp4.tmp")

with multi_count():
    with enable_counter(tempfile_path1) as counter:
        for i in counter(range(10)):
            print("1:",i)
            time.sleep(3)
            
    print("1 is finished")
    for i in simple_counter(tempfile_path2, range(5)):
            print("2:",i)
            time.sleep(3)

1 is finished
2: 2
2: 3
2: 4


### 再帰的に使う場合 

以下の例では，iが1,jが2の時にKeybordInterruptを行ったのち，再度実行したものである．

In [89]:
tempfile_path3 = Path("temp5.tmp")
tempfile_path4 = Path("temp6.tmp")

with enable_counter(tempfile_path3) as outer_counter:
    for i in outer_counter(range(5)):
        print("outer:",i)
        for j in simple_counter(tempfile_path4 ,range(5)):
                print("\tinner:",j)
                time.sleep(3)

outer: 1
	inner: 2
	inner: 3
	inner: 4
outer: 2
	inner: 0
	inner: 1
	inner: 2
	inner: 3
	inner: 4
outer: 3
	inner: 0
	inner: 1
	inner: 2
	inner: 3
	inner: 4
outer: 4
	inner: 0
	inner: 1
	inner: 2
	inner: 3
	inner: 4


###  エラーとなる処理

以下のように，`multi_count`は再帰的に利用できない

In [85]:
with multi_count():
    with multi_count():
        pass

Exception: MultiCount has already opend. cannot open another MultiCount